In [ ]:
!pip install openai
!pip install requests

In [ ]:
import openai
import requests
import os
from google.colab import userdata

In [ ]:
client = OpenAI(api_key=userdata.get('OpenAI'))
brave_api_key = userdata.get('BraveAPI')

In [ ]:
def brave_search(query):
    url = "https://api.search.brave.com/res/v1/web/search"
    headers = {
        "Accept": "application/json",
        "Accept-Encoding": "gzip",
        "X-Subscription-Token": brave_api_key
    }
    params = {
        "q": query.strip()  # Ensure the query is a simple, clean string
    }

    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        return response.json()  # Return the JSON response if the request is successful
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

def mock_search(query):
    return {
        "query": query,
        "results": [
            f"Mock result 1 for {query}",
            f"Mock result 2 for {query}",
            f"Mock result 3 for {query}"
        ]
    }

In [ ]:
# Cache for storing search results
search_cache = {}

In [ ]:
def cached_search(query, use_mock=True):
    if query in search_cache:
        print("Returning cached results")
        return search_cache[query]
    else:
        # Choose between mock search or actual API search
        if use_mock:
            results = mock_search(query)
        else:
            results = brave_search(query)

        search_cache[query] = results
        return results

In [ ]:
# Topic Breakdown Tool
def topic_breakdown_tool(main_topic):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": f"Break down the following topic into smaller subtopics or subqueries:\n\n{main_topic}"}],
        max_tokens=150
    )
    subtopics = response.choices[0].message.content.strip().split('\n')
    return [subtopic.strip() for subtopic in subtopics if subtopic.strip()]

In [ ]:
# Query Expansion Tool
def query_expansion_tool(subtopics):
    expanded_queries = []
    for subtopic in subtopics:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": f"Expand the following subtopic with related keywords, synonyms, and phrases:\n\n{subtopic}"}],
            max_tokens=100
        )
        expanded_queries.append(response.choices[0].message.content.strip())
    return expanded_queries

In [ ]:
# Critique Tool
def critique_tool(summary):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": f"Critique the following summary and provide suggestions for improvement and other relevant topics to search for:\n\n{summary}"}
        ]
    )
    critique = response.choices[0].message.content.strip()  # Extract the content from the chat completion
    return critique


In [ ]:
# Summarization Tool
def summarizer_tool(content):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": f"Summarize the following content:\n\n{content}"}
        ]
    )
    summary = response.choices[0].message.content.strip()  # Extract the content from the chat completion
    return summary

In [ ]:
def research_agent_workflow(topic, use_mock=True):
    # Step 1: Break down the topic
    subtopics = topic_breakdown_tool(topic)
    print("Subtopics:", subtopics)

    # Step 2: Expand the subqueries
    expanded_queries = query_expansion_tool(subtopics)
    print("Expanded Queries:", expanded_queries)

    # Step 3: Search for information
    if use_mock:
        search_results = mock_search(expanded_queries[0])
    else:
        simplified_query = expanded_queries[0].split("\n")[0]
        search_results = brave_search(simplified_query)

    print("Search Results:", search_results)

    # Step 4: Check and use the search results
    content_list = []
    if use_mock:
        # Use the mock results directly
        content_list.extend(search_results['results'])
    else:
        if not search_results or not isinstance(search_results, dict) or len(search_results.get("web", {}).get("results", [])) < 3:
            print("Initial search insufficient, expanding queries further.")
            further_expanded_queries = query_expansion_tool(expanded_queries)
            if use_mock:
                search_results = mock_search(further_expanded_queries[0])
            else:
                simplified_query = further_expanded_queries[0].split("\n")[0]
                search_results = brave_search(simplified_query)
            print("Refined Search Results:", search_results)

        if isinstance(search_results, dict) and "web" in search_results:
            web_results = search_results["web"].get("results", [])
            for result in web_results:
                # Look for content in multiple potential fields
                description = result.get('description', None)
                snippets = result.get('extra_snippets', [])
                title = result.get('title', None)

                # Append the available content to content_list
                if description:
                    content_list.append(description)
                if snippets:
                    content_list.extend(snippets)  # Join snippets as they may contain multiple items
                if title:
                    content_list.append(title)
        else:
            content_list.append("No relevant search results found.")

    # Join all the collected content into a single string
    content = "\n".join(content_list)
    print("Content:", content)

    # Step 6: Generate a summary
    summary = summarizer_tool(content)
    print("Summary:", summary)

    # Step 7: Critique the summary and refine if needed
    critique = critique_tool(summary)
    print("Critique:", critique)

    # Optional refinement loop
    if "improve" in critique.lower():
        summary = summarizer_tool(critique + "\n" + content)
        print("Refined Summary:", summary)

    return summary

In [ ]:
# Example 1
topic = "The impact of climate change on animals"
final_summary = research_agent_workflow(topic)
print("Final Summary:", final_summary)

Subtopics: ['1. Changes in habitat loss and fragmentation', '2. Disruption in food sources and food webs', '3. Increases in incidence and severity of extreme weather events', '4. Changes in migration patterns and species distributions', '5. Altered breeding and reproduction cycles', '6. Effects on population dynamics and species interactions', '7. Vulnerability of certain species to climate change', '8. Potential for adaptation and resilience in animal populations.']
Expanded Queries: ['- Loss of natural habitat\n- Fragmentation of ecosystems\n- Destruction of wildlife habitats\n- Disruption of ecosystems\n- Decline in biodiversity\n- Deforestation\n- Urbanization\n- Land development\n- Habitat destruction\n- Habitat degradation\n- Human impact on habitat\n- Fragmented landscapes\n- Loss of critical habitat\n- Habitat fragmentation effects\n- Habitat alteration\n- Habitat loss consequences\n- Habitat modification', '- Disruption in the food chain\n- Changes in ecosystem dynamics\n- Imp

In [ ]:
topic2 = "Impact of AI on Emergency Medicine"
final_summary2 = research_agent_workflow(topic2, use_mock=False)
print("Final Summary2:", final_summary2)

Subtopics: ['1. Use of AI in triaging patients in emergency departments', '2. AI algorithms for predicting patient outcomes in emergency situations', '3. AI-based diagnostic tools in emergency medicine', '4. AI-powered predictive analytics for resource allocation in emergency departments', '5. Ethical considerations in using AI for decision-making in emergency medicine', '6. Training and education for healthcare professionals on AI technology in emergency medicine']
Expanded Queries: ['-Artificial intelligence in patient triage \n-Implementation of AI in emergency department triage\n-Utilizing AI technology to prioritize patients \n-AI algorithms in patient assessment \n-Automatic patient prioritization using AI \n-Enhancing emergency department triage with AI \n-Application of AI in directing patient flow \n-Integrating AI systems for faster patient assessments \n-Improving efficiency in emergency department triage with AI \n-AI-driven patient prioritization in emergency care \n-Enhan

In [ ]:
topic3 = "Steps to learn LLM online"
final_summary3 = research_agent_workflow(topic3)
print("Final Summary3:", final_summary3)

Subtopics: ['1. Researching LLM programs online', '2. Choosing the right online LLM program', '3. Applying for admission to an online LLM program', '4. Preparing for online LLM classes', '5. Participating in online LLM classes', '6. Completing assignments and exams in an online LLM program', '7. Networking and building relationships with professors and classmates in online LLM programs', '8. Graduating from an online LLM program.']
Expanded Queries: ['- Evaluating online LLM programs\n- Comparing online LLM programs\n- Finding the best online LLM program\n- Exploring online LLM program options\n- Researching accredited online LLM programs\n- Investigating online LLM program curriculum\n- Online LLM program rankings\n- Online LLM program reviews\n- Online LLM program requirements\n- Online LLM program cost\n- Online LLM program admissions process\n- Online LLM program benefits\n- Online LLM program', "- Selecting the best internet Master of Laws program\n- Finding the perfect virtual LL

In [ ]:
topic4 = "Steps to learn LLM online"
final_summary4 = research_agent_workflow(topic4, use_mock=False)
print("Final Summary4:", final_summary4)

Subtopics: ['1. Researching LLM programs online', '2. Selecting a reputable online LLM program', '3. Understanding the requirements for admission', '4. Creating a study schedule and plan for completing coursework', '5. Utilizing online resources and tools for studying', '6. Participating in online discussions and virtual classes', '7. Seeking out additional help or support as needed', '8. Completing assignments and preparing for exams', '9. Engaging with faculty and peers in the online program', '10. Completing the necessary coursework to earn the LLM degree.']
Expanded Queries: ['- Exploring online LLM programs\n- Searching for online LLM degree programs\n- Investigating distance learning LLM options\n- Finding accredited online LLM programs\n- Examining online law school offerings\n- Looking into virtual LLM programs\n- Researching remote LLM degree programs \n- Comparing online law degree options\n- Discovering digital LLM programs', '- Choosing a well-established online LLM program